In [1]:
# import os + os.chir is used to get to the llms directory to use the usermodules 
import os
os.chdir('/pub/anvieyra/copilot/llms/user_modules') 

from  table_instances import Summaries, Sentiments, Base
from  db import Session, get_training_data, add_summaries_data
from helper_functions import compute_summary_scores
import pandas as pd
import matplotlib.pyplot as plt

session = Session()

In [3]:
deci = pd.read_csv("../../Datasets/deciLM_summary_results.csv")
deci.head()

,dialogue_id,summary,time_taken,memory_usage
0,1,Mr. Smith visits the doctor for a check-up. Th...,0.480078,93.707207
1,2,Dr. Peters and Mrs. Parker discuss Ricky's vac...,0.442097,91.128203
2,3,A person lost their keys and a small foot orna...,0.442097,91.128203
3,4,Person1 is upset that Person2 didn't tell them...,0.442097,91.128203
4,5,"Malik asks Wen to dance, she says no, but he t...",0.442097,91.128203


In [4]:
training_data = get_training_data()
training_data.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [5]:
merged_df = pd.merge(training_data[["dialogue_id","actual_summary"]], deci, on="dialogue_id", how='inner')
merged_df.head()

,dialogue_id,actual_summary,summary,time_taken,memory_usage
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 2 wants a suit made to measure in 100% ...,0.442097,91.128203
1,3,#Person1#'s looking for a set of keys and asks...,A person lost their keys and a small foot orna...,0.442097,91.128203
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset that Person2 didn't tell them...,0.442097,91.128203
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"Malik asks Wen to dance, she says no, but he t...",0.442097,91.128203
4,6,Lisa gives Aims a birthday present and Aims lo...,"Happy birthday, Aims! Here is a present for yo...",0.442097,91.128203


In [6]:
merged_df = merged_df.rename(columns={'summary': 'generated_summary', 'time_taken':'time_summary_taken', 'memory_usage':'memory_summary_usage'})
merged_df.head()

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 2 wants a suit made to measure in 100% ...,0.442097,91.128203
1,3,#Person1#'s looking for a set of keys and asks...,A person lost their keys and a small foot orna...,0.442097,91.128203
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset that Person2 didn't tell them...,0.442097,91.128203
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"Malik asks Wen to dance, she says no, but he t...",0.442097,91.128203
4,6,Lisa gives Aims a birthday present and Aims lo...,"Happy birthday, Aims! Here is a present for yo...",0.442097,91.128203


In [11]:
merged_df["model_id"] = "deciLM 7b"
merged_df.head()

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage,model_id
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 2 wants a suit made to measure in 100% ...,0.442097,91.128203,deciLM 7b
1,3,#Person1#'s looking for a set of keys and asks...,A person lost their keys and a small foot orna...,0.442097,91.128203,deciLM 7b
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset that Person2 didn't tell them...,0.442097,91.128203,deciLM 7b
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"Malik asks Wen to dance, she says no, but he t...",0.442097,91.128203,deciLM 7b
4,6,Lisa gives Aims a birthday present and Aims lo...,"Happy birthday, Aims! Here is a present for yo...",0.442097,91.128203,deciLM 7b


In [12]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

merged_df = merged_df.apply(compute_summary_scores, axis=1)
merged_df.head()

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage,model_id,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 2 wants a suit made to measure in 100% ...,0.442097,91.128203,deciLM 7b,0.578947,0.378378,0.578947,0.933703,0.944992,0.939314,0.743547
1,3,#Person1#'s looking for a set of keys and asks...,A person lost their keys and a small foot orna...,0.442097,91.128203,deciLM 7b,0.294118,0.051282,0.294118,0.875315,0.851999,0.863500,0.233347
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset that Person2 didn't tell them...,0.442097,91.128203,deciLM 7b,0.344828,0.142857,0.275862,0.920362,0.871602,0.895319,0.296010
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"Malik asks Wen to dance, she says no, but he t...",0.442097,91.128203,deciLM 7b,0.206897,0.000000,0.137931,0.896868,0.876806,0.886724,0.225603
4,6,Lisa gives Aims a birthday present and Aims lo...,"Happy birthday, Aims! Here is a present for yo...",0.442097,91.128203,deciLM 7b,0.125000,0.000000,0.125000,0.844445,0.892918,0.868005,0.307431


In [13]:
merged_df.to_csv("../../Datasets/deciLM_scores.csv", index = False)

In [14]:
deci_scores = pd.read_csv("../../Datasets/deciLM_scores.csv")
deci_scores.head()

,dialogue_id,actual_summary,generated_summary,time_summary_taken,memory_summary_usage,model_id,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,3816,#Person2# wants a suit made in 100 % black Bar...,Person 2 wants a suit made to measure in 100% ...,0.442097,91.128203,deciLM 7b,0.578947,0.378378,0.578947,0.933703,0.944992,0.939314,0.743547
1,3,#Person1#'s looking for a set of keys and asks...,A person lost their keys and a small foot orna...,0.442097,91.128203,deciLM 7b,0.294118,0.051282,0.294118,0.875315,0.851999,0.863500,0.233347
2,4,#Person1#'s angry because #Person2# didn't tel...,Person1 is upset that Person2 didn't tell them...,0.442097,91.128203,deciLM 7b,0.344828,0.142857,0.275862,0.920362,0.871602,0.895319,0.296010
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"Malik asks Wen to dance, she says no, but he t...",0.442097,91.128203,deciLM 7b,0.206897,0.000000,0.137931,0.896868,0.876806,0.886724,0.225603
4,6,Lisa gives Aims a birthday present and Aims lo...,"Happy birthday, Aims! Here is a present for yo...",0.442097,91.128203,deciLM 7b,0.125000,0.000000,0.125000,0.844445,0.892918,0.868005,0.307431


In [15]:
merged_df = merged_df[['dialogue_id', 'model_id', 'generated_summary', 'rouge_1', 'rouge_2', 'rouge_l', 'bert_precision', 'bert_recall', 'bert_f1', 'meteor', 'memory_summary_usage', 'time_summary_taken']]
merged_df.head()

,dialogue_id,model_id,generated_summary,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor,memory_summary_usage,time_summary_taken
0,3816,deciLM 7b,Person 2 wants a suit made to measure in 100% ...,0.578947,0.378378,0.578947,0.933703,0.944992,0.939314,0.743547,91.128203,0.442097
1,3,deciLM 7b,A person lost their keys and a small foot orna...,0.294118,0.051282,0.294118,0.875315,0.851999,0.863500,0.233347,91.128203,0.442097
2,4,deciLM 7b,Person1 is upset that Person2 didn't tell them...,0.344828,0.142857,0.275862,0.920362,0.871602,0.895319,0.296010,91.128203,0.442097
3,5,deciLM 7b,"Malik asks Wen to dance, she says no, but he t...",0.206897,0.000000,0.137931,0.896868,0.876806,0.886724,0.225603,91.128203,0.442097
4,6,deciLM 7b,"Happy birthday, Aims! Here is a present for yo...",0.125000,0.000000,0.125000,0.844445,0.892918,0.868005,0.307431,91.128203,0.442097


In [16]:
add_summaries_data(merged_df)